In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import mlflow
import json

def generate_transaction_data(n_samples=40):
    np.random.seed(42)
    start_date = datetime(2024, 12, 12)
    
    data = []
    
    for i in range(n_samples):
        tx_datetime = start_date + timedelta(
            days=np.random.randint(0, 30),
            hours=np.random.randint(0, 24),
            minutes=np.random.randint(0, 60),
            seconds=np.random.randint(0, 60)
        )
        
        time_seconds = tx_datetime.hour *  3600 + tx_datetime.minute * 60 + tx_datetime.second
        time_days = tx_datetime.day
        
        record = {
            "TRANSACTION_ID": int(i + 1000),
            "TX_DATETIME": tx_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            "CUSTOMER_ID": int(np.random.randint(500, 700)),
            "TERMINAL_ID": int(np.random.randint(100, 200)),
            "TX_AMOUNT": float(round(np.random.uniform(10, 1000), 2)),
            "TX_TIME_SECONDS": int(time_seconds),
            "TX_TIME_DAYS": int(time_days)
        }
        data.append(record)
    
    return pd.DataFrame(data)

# Generate simulated transactions
simulated_data = generate_transaction_data(40)

# Load model
logged_model = 'runs:/4ce29c8a57f945639a4e5fb57ec1a32a/fraud_detection_inference'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [0]:
simulated_data.head()

In [0]:
loaded_model.predict(simulated_data.iloc[[15]])

In [0]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/b969b933617f46f9bd131fbe86d9b181/fraud_detection_inference'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": {
    "TRANSACTION_ID": 4781,
    "TX_DATETIME": "2024-10-29 12:57:40",
    "CUSTOMER_ID": 456,
    "TERMINAL_ID": 139,
    "TX_AMOUNT": 20.25,
    "TX_TIME_SECONDS": 46660,
    "TX_TIME_DAYS": 29
  }
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)